# Aggregate demographics

This is similar to aggregate_responses but performed separately as we want to aggregate for one school and then v.s. all other schools as one group - and we don't want to breakdown those results any further by any demographic characteristics.

**Note:**
* This method assumes that all pupils in the dataset belong to a school, and that the dataset only contains Northern Devon schools. If either of those conditions changes, you will need to review the code below.

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from collections import defaultdict
from dataclasses import dataclass
import numpy as np
import os
import pandas as pd

# Import functions from our dashboard package
from kailo_beewell_dashboard.response_labels import (
    create_symbol_response_label_dict)
from kailo_beewell_dashboard.create_and_aggregate_data import (
    aggregate_demographic)

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Stores paths to data and files'''
    data = '../data/survey_data'
    synthetic_data = 'synthetic_data_raw.csv'
    aggregate = 'aggregate_demographic.csv'


paths = Paths()

### Import raw data

In [3]:
data = pd.read_csv(os.path.join(paths.data, paths.synthetic_data))
data.head()

,symbol_family,symbol_home,symbol_friends,symbol_choice,symbol_things,symbol_health,symbol_future,symbol_school,symbol_free,symbol_life,...,symbol_school_lab,symbol_free_lab,symbol_life_lab,gender_lab,year_group_lab,fsm_lab,sen_lab,ethnicity_lab,english_additional_lab,school_lab
0,2.0,2,1.0,1.0,NaN,1,1.0,2.0,2.0,3.0,...,Ok,Ok,Sad,NaN,Year 11,FSM,SEN,Ethnic minority,Yes,School B
1,2.0,1,1.0,2.0,2.0,3,2.0,3.0,2.0,1.0,...,Sad,Ok,Happy,NaN,Year 10,Non-FSM,NaN,Ethnic minority,Yes,School A
2,2.0,1,2.0,2.0,2.0,1,2.0,2.0,1.0,1.0,...,Ok,Happy,Happy,NaN,Year 11,FSM,Non-SEN,White British,Yes,School B
3,1.0,2,3.0,3.0,2.0,1,2.0,1.0,NaN,3.0,...,Happy,NaN,Sad,Female,Year 11,Non-FSM,SEN,NaN,Yes,School B
4,3.0,1,NaN,2.0,2.0,2,NaN,2.0,NaN,1.0,...,Ok,NaN,Happy,Female,Year 10,FSM,NaN,White British,No,School B


## Aggregate data

In [4]:
# Make list of demographic columns
response_col = [
    'gender', 'year_group', 'fsm', 'sen', 'ethnicity', 'english_additional']

# Import dictionary which has response options for each variable
labels = create_symbol_response_label_dict()

# Add 'NaN': 'No data' to the dictionary
for col in response_col:
    labels[col].update({np.nan: 'No data'})

# Preview two examples
print(labels['fsm'])
print(labels['year_group'])

{0: 'Non-FSM', 1: 'FSM', nan: 'No data'}
{7: 'Year 7', 8: 'Year 8', 9: 'Year 9', 10: 'Year 10', 11: 'Year 11', nan: 'No data'}


In [5]:
# Aggregated demographic columns by school (v.s. all other schools)
result = aggregate_demographic(data, response_col, labels)
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,school_group,school_group_lab
0,"[0, 1, nan]","[Male, Female, No data]","[12, 14, 4]","[40.0, 46.666666666666664, 13.333333333333334]",gender,30.0,School A,1,Your school
0,"[7, 8, 9, 10, 11, nan]","[Year 7, Year 8, Year 9, Year 10, Year 11, No ...",NaN,NaN,year_group,NaN,School A,1,Your school
0,"[0, 1, nan]","[Non-FSM, FSM, No data]","[13, 17, 0]","[43.333333333333336, 56.666666666666664, 0.0]",fsm,30.0,School A,1,Your school
0,"[0, 1, nan]","[Non-SEN, SEN, No data]","[14, 13, 3]","[46.666666666666664, 43.333333333333336, 10.0]",sen,30.0,School A,1,Your school
0,"[1, 2, nan]","[Ethnic minority, White British, No data]","[19, nan, nan]","[63.33333333333333, nan, nan]",ethnicity,19.0,School A,1,Your school


## Add labels for each measure

In [6]:
# Define labels
labels = {
    'gender': 'Gender',
    'year_group': 'Year group',
    'fsm': 'Free school meals',
    'sen': 'Special educational needs',
    'ethnicity': 'Ethnicity',
    'english_additional': 'English as an additional language'
}

# Add labels
result['measure_lab'] = result['measure'].map(labels)

# Preview
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,school_group,school_group_lab,measure_lab
0,"[0, 1, nan]","[Male, Female, No data]","[12, 14, 4]","[40.0, 46.666666666666664, 13.333333333333334]",gender,30.0,School A,1,Your school,Gender
0,"[7, 8, 9, 10, 11, nan]","[Year 7, Year 8, Year 9, Year 10, Year 11, No ...",NaN,NaN,year_group,NaN,School A,1,Your school,Year group
0,"[0, 1, nan]","[Non-FSM, FSM, No data]","[13, 17, 0]","[43.333333333333336, 56.666666666666664, 0.0]",fsm,30.0,School A,1,Your school,Free school meals
0,"[0, 1, nan]","[Non-SEN, SEN, No data]","[14, 13, 3]","[46.666666666666664, 43.333333333333336, 10.0]",sen,30.0,School A,1,Your school,Special educational needs
0,"[1, 2, nan]","[Ethnic minority, White British, No data]","[19, nan, nan]","[63.33333333333333, nan, nan]",ethnicity,19.0,School A,1,Your school,Ethnicity


## Save results to CSV

In [7]:
# Show the data types and presence of null
types = result.dtypes
null = result.isnull().any()
pd.DataFrame([types, null]).T

,0,1
cat,object,False
cat_lab,object,False
count,object,True
percentage,object,True
measure,object,False
n_responses,float64,True
school_lab,object,False
school_group,int64,False
school_group_lab,object,False
measure_lab,object,False


In [8]:
result.to_csv(os.path.join(paths.data, paths.aggregate),
              index=False, na_rep='NULL')